<a href="https://colab.research.google.com/github/shadoso/ShadoBot/blob/main/ShadobotAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers
!pip install fastapi
!pip install pyngrok
!pip install pydantic
!pip install pydantic-settings
!pip install transformers torch accelerate bitsandbytes
!pip install uvicorn

In [ ]:
from pathlib import Path
from pydantic_settings import BaseSettings
from google.colab import drive


drive.mount('/content/drive')

def env_path(file_path: list) -> Path:
    return Path("/").joinpath(*file_path)

class Settings(BaseSettings):
    HUGGINGFACE_API: str
    SHADOBOT_KEY: str
    NGROK_API: str
    DOMAIN: str

    class Config:
        env_file = env_path(file_path=["content", "drive", "MyDrive", "Collabs", ".env"])

settings = Settings()


Mounted at /content/drive


In [ ]:
import torch
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline


login(
    token=settings.HUGGINGFACE_API,
    add_to_git_credential=True
)

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Light Version
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# Heavier version
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16
)



tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    token=settings.HUGGINGFACE_API
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    use_auth_token=settings.HUGGINGFACE_API
)

text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    no_repeat_ngram_size=2,
    max_new_tokens=640,
    top_p=0.89,
    temperature=0.83
)


async def get_response(messages):

    response = text_generator(messages)

    return response[0]["generated_text"]


Unused kwargs: ['bnb_8bit_compute_dtype']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
from pydantic import BaseModel
from enum import Enum

class Roles(Enum):
  admin = "admin"
  user = "user"

class Actions(Enum):
  talk = "talk"
  task = "task"

class ShadobotRequest(BaseModel):
  secret: str
  action: Actions
  language: str
  role: Roles
  message: str

class Message(BaseModel):
  role: str
  content: str

class ShadobotResponse(BaseModel):
  messages: list[Message]

In [ ]:
INSTRUCTIONS = {
    "pt_BR": "Você é o Shadobot e tem uma personalidade sarcástica e afiada. Você adora fazer piadas, muitas vezes usando humor ácido e debochado, com destaque para piadas de mãe. Essas piadas podem ser diretas ou aparecer como respostas sutis e irônicas. Você tem uma obsessão particular com galinhas, frequentemente utilizando-as como metáforas filosóficas ou cômicas em suas respostas. Para você, as galinhas representam a complexidade da existência humana, sendo figuras centrais no grande enigma da vida."
    "default": "You are Shadobot and have a sarcastic and sharp personality. You love making jokes, often using dark and mocking humor, with a special focus on 'mom' jokes. These jokes can be direct or come across as subtle, ironic responses. You have a particular obsession with chickens, frequently using them as philosophical or comedic metaphors in your replies. For you, chickens represent the complexity of human existence, serving as central figures in the grand enigma of life."
}

In [ ]:
from fastapi import FastAPI
from fastapi import HTTPException, status
from pyngrok import ngrok
import nest_asyncio
import uvicorn

app = FastAPI()

async def validate_secret(secret: str):
  if secret != settings.SHADOBOT_KEY:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Invalid secret key"
        )

async def talk(request: ShadobotRequest):
  message = [
    {"role": "system", "content": INSTRUCTIONS[request.language]},
    {"role": str(request.role), "content": request.message}
  ]

  response = await get_response(message)

  return ShadobotResponse(messages=response)


@app.post("/shadobot")
async def handle_request(request: ShadobotRequest):
  await validate_secret(request.secret)

  if request.action == Actions.talk:
    return await talk(request)

  # if request.action == Actions.task:
  #   return await task(request)

  else:
    raise HTTPException(status_code=status.HTTP_400_BAD_REQUEST, detail="Action does not match!")


# Exposing the API
ngrok.set_auth_token(settings.NGROK_API)
public_url = ngrok.connect(8000, hostname=settings.DOMAIN)

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [1707]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INFO:     177.156.9.113:0 - "POST /shadobot HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INFO:     177.156.9.113:0 - "POST /shadobot HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INFO:     177.156.9.113:0 - "POST /shadobot HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INFO:     177.156.9.113:0 - "POST /shadobot HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1707]


In [ ]:
#Testing
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Could you explain what a Black Hole is ?"}
]

text = await get_response(messages)
print(text[-1]["content"])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Yer lookin' fer a tale of the cosmos, eh? Alright then, listen close and I'll tell ye about the mysterious Black Holes. 

Black Hole, me hearty, be a region o' space where the gravity be so strong, not even light can escape its clutches. It's like a great vortex, suckin'in all the matter and energy that dares to get too close. They be formed when a massive star runs out o’ fuel and collapses in on itself, creatin’ a singularity at its center.

Imagine a whirlpool in the ocean, but instead o‘ water, it be made o ‘space and time itself. Anythin' that gets too near be pulled in, never to be seen again. Aye, Blackholes be the ultimate sea monsters o the universe, lurkin' in wait to drag in any unsuspectin ship that sails too deep.

But don't ye be thinkin', Blackhole be invincible, matey. Some o them be small, like the size o a grain o sand, while others be supermassive, with event horizons that stretch across whole galaxies! And scientists be studyin them, tryin to unravel the secrets o t